# 7章 マルチクラス分類

In [2]:
import os
import glob
import json
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertModel
import pytorch_lightning as pl

本章では、文章のマルチクラス分類を行う。マルチラベル分類は、複数のカテゴリに所属する文章を分類するタスクである。<br>
ラベルには、Multi-Hotベクトルと呼ばれるベクトルをあてる。これは、$(0, 1, 1, 0)$といった、One-Hotベクトルの発展的なかたちで、各要素(ラベル)に対しそれぞれフラグがたったものである。$(0, 1, 1, 0)$は、ある文章が4つのカテゴリのうちカテゴリ2と3にあてはまることを意味する。

まずは`transformers.BertModel`クラスをベースに`BertForSequenceClassificationMultiLabel`クラスを実装し、その挙動を確認していく。実装に関して、シングルラベル分類とはいくつか異なる点が存在するため注意する。

1. 3つ以上のカテゴリを持つシングルラベル分類と違い、損失関数には`BinaryCrossEntropyLoss`を用いる。<br>
Multi-Hotベクトルは各カテゴリに対して`0 or 1`で表現されるためである。
2. 各カテゴリ(Multi-Hotベクトルの各要素)に対し文章が当てはまる確率を出力するため、>50%の際に1とする実装が必要である。

本章では、モデルの最終層の出力をすべてのトークンに対し平均化し、線形変換を適用したものをスコアとする。<br>
平均化にあたり、文章長を調整する`[PAD]`トークンを削除する必要がある。`[PAD]`トークンは`encoding`で得られる`attention_mask`で`0`が与えられるため、`attention_mask`が`1`のトークンで平均を取るようにする。

In [17]:
# 7-4

class BertForSequenceClassificationMultiLabel(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()

        # BERTモデルの読み込み
        self.model = BertModel.from_pretrained(model_name)
        # 線形結合の初期化
        self.linear = torch.nn.Linear(
            self.model.config.hidden_size, num_labels
        )
    
    def forward(
        self, 
        input_ids: torch.Tensor=None, 
        attention_mask: torch.Tensor=None, 
        token_type_ids: torch.Tensor=None,
        labels: torch.Tensor=None,
        ):
        # モデルの最終層の出力
        model_output = self.model(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids,
        )
        # (batch_size, トークン数, 隠れ層数768) のtorch.Tensorを得る
        # ここでは(2, 17, 768)
        last_hidden_state = model_output.last_hidden_state

        # [PAD]トークン以外で平均を取る
        # attention_maskはtokenizerより渡された
        # (バッチサイズ, トークン数) のOne-Hotベクトルのtorch.Tensor
        # ここでは(2, 17)
        # torch.Tensor.unsqueeze(axis)メソッドで新規に1の次元を挿入する
        # (2, 17) -> (2, 17, 1)
        # torch.Tensor.unsqueeze(axis=2)と同義
        # last_hidden_stateが3次元ベクトルなので合わせる
        # torch.Tensor.sum(axis)で次元に沿って2次元目で足し算する
        # (2, 17, 768) -> (2, 768)
        # attention_mask.sum(axis=1)でattention_maskが1となる数を取得する
        # keepdim=Trueにすることで、sum()で次元が(2, 17) -> (2,)になるのを防ぎ
        # 2次元テンソルの形 今回は(2, 1) を維持する
        averaged_hidden_state = (
            last_hidden_state * attention_mask.unsqueeze(axis=-1)
        ).sum(axis=1) / attention_mask.sum(axis=1, keepdim=True)

        # 線形結合する
        scores = self.linear(averaged_hidden_state)

        # lossの計算
        output = {"logits": scores}
        if labels is not None:
            loss = torch.nn.BCEWithLogitsLoss()(scores, labels.float()) # float型に変換する必要がある
            output["loss"] = loss

        # 属性でアクセスできるようにする
        # lossにはoutput.lossでアクセスできるようになる
        output = type("bert_output", (object,), output)
        return output

<br>

モデルと文章、ラベルを定義する。

In [ ]:
# 7-5, 7-6

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=2
)
# model = mode.cuda()

# [CLS] + 本文15トークン + [SEP] の最大17トークン
# ['今日', 'の', '仕事', 'は', 'うまく', 'いっ', 'た', 'が', '、', '体調', 'が', 'あまり', '良く', 'ない', '。']
text_lst = [
    '今日の仕事はうまくいったが、体調があまり良くない。', 
    '昨日は楽しかった。'
]

# 2次元目は[負の感情, 正の感情]
labels_lst = [
    [1, 1],
    [0, 1]
]

<br>

トークナイザを定義し予測を行う。

In [23]:
encoding = tokenizer(text_lst, padding='longest', return_tensors='pt')
encoding["labels"] = torch.tensor(labels_lst)
# encoding = {key: torch.tensor(value) for key, value in encoding.items()}

output = model(**encoding)
scores = output.logits
labels_pred = (scores > 0).int() # スコアが>0ならTrue、そうでないならFalse
accuracy = accuracy_score(labels.numpy(), labels_pred.numpy())

In [50]:
print(f"y_true: {labels}")
print(f"y_pred: {labels_pred}")
print(f"accuracy: {accuracy}")
print(f"loss: {output.loss:.3f}")

y_true: tensor([[1, 1],
        [0, 1]])
y_pred: tensor([[0, 1],
        [0, 1]], dtype=torch.int32)
accuracy: 0.5
loss: 0.681


1文目はラベル`[1, 1]`に対し`[0, 1]`と予測してハズレ<br>
2文目はラベル`[0, 1]`に対し`[0, 1]`と予測してアタリ<br>
総合でAccuracyは0.5となる。

## 7.5 chABSA-datasetでマルチクラス分類

本章では、TIS株式会社が公開している、[上場企業の有価証券報告書から作成されたマルチラベルデータセット`chABSA-dataset`](https://github.com/chakki-works/chABSA-dataset)を用いる。

このデータセットは、「ネガティブ」「ポジティブ」「ニュートラル」という3クラスを用いる。文章に対してそれぞれのクラスに該当する表現があると、カテゴリーと何を対象としているかをラベル付けしている。

In [52]:
os.makedirs("data/chapter7", exist_ok=False)

In [54]:
!curl --output "data/chapter7/chABSA-dataset.zip" "https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/chABSA-dataset.zip"
!unzip -q -d "data/chapter7" "data/chapter7/chABSA-dataset.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  705k  100  705k    0     0  1524k      0 --:--:-- --:--:-- --:--:-- 1541k
Archive:  data/chapter7/chABSA-dataset.zip
   creating: chABSA-dataset/
  inflating: chABSA-dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/chABSA-dataset/
  inflating: __MACOSX/chABSA-dataset/._.DS_Store  
 extracting: chABSA-dataset/.gitkeep  
  inflating: chABSA-dataset/e00008_ann.json  
  inflating: chABSA-dataset/e00017_ann.json  
  inflating: chABSA-dataset/e00024_ann.json  
  inflating: chABSA-dataset/e00026_ann.json  
  inflating: chABSA-dataset/e00030_ann.json  
  inflating: chABSA-dataset/e00033_ann.json  
  inflating: chABSA-dataset/e00034_ann.json  
  inflating: chABSA-dataset/e00035_ann.json  
  inflating: chABSA-dataset/e00037_ann.json  
  inflating: chABSA-dataset/e00051_ann.json  
  inflating: chABSA-dataset/e00053_ann.j

それぞれのファイルは`chABSA-dataset/e*****_ann.json`で、全体で230ある。

In [ ]:
# 7-9
data = json.load(open('chABSA-dataset/e00030_ann.json'))
print( data['sentences'][0] )

In [ ]:
# 7-10
category_id = {'negative':0, 'neutral':1 , 'positive':2}

dataset = []
for file in glob.glob('chABSA-dataset/*.json'):
    data = json.load(open(file))
    # 各データから文章（text）を抜き出し、ラベル（'labels'）を作成
    for sentence in data['sentences']:
        text = sentence['sentence'] 
        labels = [0,0,0]
        for opinion in sentence['opinions']:
            labels[category_id[opinion['polarity']]] = 1
        sample = {'text': text, 'labels': labels}
        dataset.append(sample)

In [ ]:
# 7-11
print(dataset[0])

In [ ]:
# 7-12
# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for sample in dataset:
    text = sample['text']
    labels = sample['labels']
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True
    )
    encoding['labels'] = labels
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

# データセットの分割
random.shuffle(dataset_for_loader) 
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

#　データセットからデータローダを作成
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [ ]:
# 7-13
class BertForSequenceClassificationMultiLabel_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters() 
        self.bert_scml = BertForSequenceClassificationMultiLabel(
            model_name, num_labels=num_labels
        ) 

    def training_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_scml(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)

    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels')
        output = self.bert_scml(**batch)
        scores = output.logits
        labels_predicted = ( scores > 0 ).int()
        num_correct = ( labels_predicted == labels ).all(-1).sum().item()
        accuracy = num_correct/scores.size(0)
        self.log('accuracy', accuracy)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

trainer = pl.Trainer(
    gpus=1, 
    max_epochs=5,
    callbacks = [checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, 
    num_labels=3, 
    lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
# 7-14
# 入力する文章
text_list = [
    "今期は売り上げが順調に推移したが、株価は低迷の一途を辿っている。",
    "昨年から黒字が減少した。",
    "今日の飲み会は楽しかった。"
]

# モデルのロード
best_model_path = checkpoint.best_model_path
model = BertForSequenceClassificationMultiLabel_pl.load_from_checkpoint(best_model_path)
bert_scml = model.bert_scml.cuda()

# データの符号化
encoding = tokenizer(
    text_list, 
    padding = 'longest',
    return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() }

# BERTへデータを入力し分類スコアを得る。
with torch.no_grad():
    output = bert_scml(**encoding)
scores = output.logits
labels_predicted = ( scores > 0 ).int().cpu().numpy().tolist()

# 結果を表示
for text, label in zip(text_list, labels_predicted):
    print('--')
    print(f'入力：{text}')
    print(f'出力：{label}')